In [1]:
%load_ext autoreload

%autoreload 2

from RNN2DFA.LSTM import LSTMNetwork
# from GRU import GRUNetwork
from RNN2DFA.RNNClassifier import RNNClassifier
from RNN2DFA.Training_Functions import mixed_curriculum_train
import Tomita_Grammars 
from RNN2DFA.Training_Functions import make_test_set,make_train_set_for_target
from RNNexplainer import Explainer
import pandas as pd
import LTL2DFA as ltlf2dfa





In [2]:
from specific_examples import Alternating_Bit_Protocol
abp=Alternating_Bit_Protocol()
generator_dfa=abp.dfa
target_formula=abp.target_formula
alphabet=generator_dfa.alphabet



In [3]:
from specific_examples import Email
email=Email()
generator_dfa=email # to make everything work
target_formula=email.target_formula
alphabet=email.alphabet


In [4]:



# make training set on grammars
# target = Tomita_Grammars.tomita_email
# alphabet = "pqradmn"
# generator_dfa.classify_word=target
# target_formula="mail string match"


# use a dfa to generate training set
target_formula="F(a & X(b))"
alphabet="abc"
import LTL2DFA as ltlf2dfa
generator_dfa=ltlf2dfa.translate_ltl2dfa(alphabet=[character for character in alphabet],formula=target_formula)
print(generator_dfa)





# print(train_set)
# print("\n\nPositive examples:")
# for key in train_set:
#     if(train_set[key]):
#         print(key)
# print(train_set)


DFA:->
 - alphabet: ['a', 'b', 'c']
 - Q: [1, 2, 3, 4]
 - q0: 1
 - F: [4]
 - delta: {1: {'000': 2, '001': 2, '010': 2, '011': 2, '100': 3, '101': 3, '110': 3, '111': 3}, 2: {'000': 2, '001': 2, '010': 2, '011': 2, '100': 3, '101': 3, '110': 3, '111': 3}, 3: {'000': 2, '001': 2, '010': 4, '011': 4, '100': 3, '101': 3, '110': 4, '111': 4}, 4: {'000': 4, '001': 4, '010': 4, '011': 4, '100': 4, '101': 4, '110': 4, '111': 4}}
 - is_singleton_graph: False


In [5]:
fout=open("output/log.txt", "a")
fout.write(".........................................................................\n")
fout.write("Target: "+ target_formula)
fout.write("\n")
fout.close()

In [6]:
# make training set
train_set = make_train_set_for_target(generator_dfa.classify_word,alphabet,max_train_samples_per_length=1000,search_size_per_length=3000)
# print(train_set)

made train set of size: 10860 , of which positive examples: 5427


In [7]:
print(train_set)

bbcccc': False, 'acbccbbbcccacbbacbba': False, 'bbcbcbaaacbcbcaccaca': False, 'cbcacbacbaccccbaacba': False, 'cbbccbcbccaacaaacaca': False, 'bbbbcbacccaaacaacacc': False, 'ccaacbcbcbacbcacacbb': False, 'cbccbcccbcccacacbacb': False, 'ccbcccbccbcacbacccbb': False, 'accaccbcccbbaaacbaac': False, 'bcaaacbbbaccaaccaaca': False, 'bbcbcbacaacbbcccbacb': False, 'bacacbcaccaaccaaacaa': False, 'accacaaccbcccbacbcba': False, 'cacacbcbbcccccbbbbbc': False, 'aacbbcbcbbacaaaccbbc': False, 'bbbbbccbacccbbcbcccc': False, 'aacbbbbacbcacaacacbb': False, 'cacbcaaccccaaacbbcac': False, 'bbcbacbaccacccaccbcb': False, 'bcacaaaacbcaaacbcbac': False, 'cbcccbcbacaaacaaacbb': False, 'caacbcaacbbbccbccbbc': False, 'bbaaacbbbaacacbbaccc': False, 'cbbacacbbbccbcccbcbc': False, 'accbcaaacbccacccbaca': False, 'cbccbbccaaacbacccbba': False, 'bbcbbcbcaccacbcbbaac': False, 'baacaccacbbbbbaacaca': False, 'bccbcacaccccbcacaaaa': False, 'caaccbaacbcbcbcacaca': False, 'cbaccbbcbbcbccccbccb': False, 'cbbbbcaccaaccbbbbbca':

In [8]:
print(len(train_set))
if(target_formula=="email match"):
    matching_strings=email.generate_matching_strings(1000,20)
    for string in matching_strings:
        train_set[string]=True
print(len(train_set))

10860
10860


In [17]:


# define rnn
rnn = RNNClassifier(alphabet,num_layers=1,hidden_dim=10,RNNClass = LSTMNetwork)


# train the model
mixed_curriculum_train(rnn,train_set,stop_threshold = 0.0005)
rnn.renew()  
dfa_from_rnn=rnn 
# statistics

def percent(num,digits=2):
    tens = pow(10,digits)
    return int(100*num*tens)/tens

test_set = train_set 
print("testing on train set, i.e. test set is train set")
# we're printing stats on the train set for now, but you can define other test sets by using
# make_train_set_for_target

n = len(test_set)
print("test set size:", n)
pos = 0
rnn_target = 0
for w in test_set:
    if generator_dfa.classify_word(w):
        pos+=1

    if dfa_from_rnn.classify_word(w)==generator_dfa.classify_word(w):
        rnn_target+=1
print("rnn score against target on test set:                             ",rnn_target,"("+str(percent(rnn_target/n))+")")

# dfa_from_rnn=generator_dfa

classification loss on last batch was: 0.00046116977508358254
testing on train set, i.e. test set is train set
test set size: 10860
rnn score against target on test set:                              10860 (100.0)


In [11]:


# use a query LTL formula
query_formula="true"
query_dfa=ltlf2dfa.translate_ltl2dfa(alphabet=[character for character in alphabet],formula=query_formula)

"""  
Create initial samples
"""


from RNNexplainer import Traces
traces=Traces(rnn, alphabet)
traces.label_from_network([])
traces.write_in_file()




from PACTeacher.pac_teacher import PACTeacher as Teacher 
explainer=Explainer(alphabet=[character for character in alphabet])
teacher = Teacher(dfa_from_rnn,epsilon=.05, delta=.05, max_trace_length=20, max_formula_depth=10, query_dfa=query_dfa)



import time
start_time=time.time()
from multiprocessing import Process, Queue
explainer, flag= teacher.teach(explainer, traces, timeout = 15)
end_time=time.time()


print("\n\nepsilon=", teacher.epsilon, "delta=", teacher.delta, "max_trace_length=", teacher.max_trace_length)
print("query:", query_formula)
print("final ltl: ", explainer.ltl)

fout=open("output/log.txt", "a")
fout.write("\n\nquery: "+query_formula)
fout.write("\nfinal LTL: "+ explainer.ltl)
if(not flag):
    fout.write(" [incomplete]")
    print("incomplete formula")
fout.write("\n\n")

print("\nTime taken:", end_time-start_time)
fout.close()


test_set = train_set 
fout=open("output/log.txt", "a")
# fout.write("rnn score against target on test set:                             "+str(rnn_target)+"("+str(percent(rnn_target/n))+")")
# fout.write("\n")

performance_ltl_wo_query = performance_ltl_with_target_wo_query = performance_ltl = performance_ltl_with_target = 0


for w in test_set:
    if dfa_from_rnn.classify_word(w)==explainer.dfa.classify_word(w):
        performance_ltl_wo_query+=1
    if explainer.dfa.classify_word(w)==generator_dfa.classify_word(w):
        performance_ltl_with_target_wo_query +=1
    if (dfa_from_rnn.classify_word(w)and query_dfa.classify_word(w)) ==explainer.dfa.classify_word(w):
        performance_ltl+=1
    if explainer.dfa.classify_word(w)== (generator_dfa.classify_word(w) and query_dfa.classify_word(w)):
        performance_ltl_with_target +=1

print("extracted LTL score against rnn on test set:                      ",performance_ltl_wo_query,"("+str(percent(performance_ltl_wo_query/n))+")")

print("extracted LTL score against target on rnn's test set:             ",performance_ltl_with_target_wo_query,"("+str(percent(performance_ltl_with_target_wo_query/n))+")")

print("extracted LTL score against rnn on test set (with query):         ",performance_ltl,"("+str(percent(performance_ltl/n))+")")

print("extracted LTL score against target on rnn's test set (with query):",performance_ltl_with_target,"("+str(percent(performance_ltl_with_target/n))+")")

fout.close()


# report in a pandas file
result = pd.DataFrame(columns=['target', 
                                'query', 
                                'explanation', 
                                'status', 
                                'rnn score', 
                                'explanation score', 
                                'explanation score on ground truth',
                                'extraction time'
                                ])

result = result.append(
    {
        'target':target_formula,
        'query':query_formula,
        'explanation':explainer.ltl,
        'status':flag,
        'rnn score':percent(rnn_target/n),
        'explanation score':percent(performance_ltl/n),
        'explanation score on ground truth':percent(performance_ltl_with_target/n),
        'extraction time': end_time-start_time
    }, ignore_index=True
)
print(result.to_string(index=False))
result.to_csv('output/result.csv', header=False, index=False, mode='a')




positive traces---> 
[]


negative traces---> 
[]



learned LTL formula: true
Learning took:  0.052208900451660156  s
EQ test took  0.0021071434020996094  s
new counterexample:   should be rejected by implementation


positive traces---> 
[]


negative traces---> 
['']



0  iteration complete



learned LTL formula: false
Learning took:  0.059403181076049805  s
EQ test took  0.03915596008300781  s
new counterexample: ab  should be accepted by implementation


positive traces---> 
['ab']


negative traces---> 
['']



1  iteration complete



start formula depth: 1
learned LTL formula: a
Learning took:  0.0794057846069336  s
EQ test took  0.008790969848632812  s
new counterexample: a  should be rejected by implementation


positive traces---> 
['ab']


negative traces---> 
['', 'a']



2  iteration complete



start formula depth: 1
learned LTL formula: (X b)
Learning took:  0.12059330940246582  s
EQ test took  0.013554811477661133  s
new counterexample: bab  should be accepted by i

In [8]:
# read result
# df=pd.read_csv("output/result.csv",header=None)
# df.columns=['target', 
#             'query', 
#             'explanation', 
#             'status', 
#             'rnn score', 
#             'explanation score', 
#             'explanation score on ground truth',
#             'extraction time'
#             ]
# print(df.to_string(index=False))